In [1]:
# import packages
import numpy as np
import pandas as pd

In [2]:
# read and combine data files 
df = pd.concat(map(pd.read_csv, ['data\Vehicle Population - 2021 Passenger Vehicles_Full_data.csv', 'data\Vehicle Population - 2021 Commercial Vehicles_Full_data.csv']))
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2018182 entries, 0 to 777980
Data columns (total 16 columns):
 #   Column                       Dtype 
---  ------                       ----- 
 0   Veh Pop - Criteria Selector  object
 1   Vehicle Use                  object
 2   Body Style                   object
 3   Electric Vehicle Flag        object
 4   Fleet Flag                   object
 5   Hybrid Vehicle Flag          object
 6   Make                         object
 7   Model                        object
 8   Model Year                   int64 
 9   Municipality                 object
 10  Person Org Type              object
 11  Region                       object
 12  Vehicle Count Year           int64 
 13  Vehicle Count                int64 
 14  Anti Theft Device Indicator  object
 15  Fuel Type                    object
dtypes: int64(3), object(13)
memory usage: 261.8+ MB


In [3]:
# drop columns
df = df[['Electric Vehicle Flag', 'Hybrid Vehicle Flag', 'Make', 'Model', 'Model Year', 'Body Style', 'Vehicle Count']]
df = df.groupby(['Electric Vehicle Flag', 'Hybrid Vehicle Flag', 'Make', 'Model', 'Model Year', 'Body Style']).aggregate('sum').reset_index()

In [4]:
def get_acura(df):
    df = pd.DataFrame(df)

    # filter vehicles from Acura
    acura = df.loc[(df['Make'].str.match('ACURA'))].copy()
    acura['Make'] = 'ACURA'

    # remove whitespace in model name
    acura.loc[(acura['Model'].str.match(r'\d\.\d .')), 'Model'] = acura.loc[(acura['Model'].str.match(r'\d\.\d .')), 'Model'].str.replace(' ', '', 1)

    # split model name and trim
    acura['Trim'] = acura['Model'].str.split(' ', n=1, expand=True)[1].str.strip()
    acura['Model'] = acura['Model'].str.split(' ', n=1, expand=True)[0].str.strip()

    # adjust model name
    acura.loc[(acura['Model'] == '1.7EL') & ((acura['Model Year'] == 2004) | (acura['Model Year'] == 2005)), 'Model'] = 'EL'
    acura.loc[(acura['Model'] == '3.2TL') & (acura['Model Year'] == 2004), 'Model'] = 'TL'
    acura.loc[(acura['Model'] == 'TL') & (acura['Model Year'] == 2015), 'Model'] = 'TLX'
    acura.loc[(acura['Model'] == 'NSX-T'), 'Model'] = 'NSX'

    # extract and remove driven wheels from trim
    acura['Drive'] = acura['Trim'].str.extract(r'((2|4|A|F|R)(WD|WHDR))')[0]
    acura['Trim'] = acura['Trim'].str.replace(r'((2|4|A|F|R)(WD|WHDR))', '', regex=True)

    # extract and remove number of doors from trim
    acura['Door'] = acura['Trim'].str.extract(r'(\dDR)')
    acura['Trim'] = acura['Trim'].str.replace(r'(\dDR)', '', regex=True)

    # extract number of doors from body style
    acura.loc[(acura['Door'].isna()), 'Door'] = acura.loc[acura['Door'].isna(), 'Body Style'].str.extract(r'((Two|Four)Door)')[0]
    acura.loc[(acura['Door'] == 'TwoDoor'), 'Door'] = '2DR'
    acura.loc[(acura['Door'] == 'FourDoor'), 'Door'] = '4DR'

    # impute driven wheels
    acura.loc[(acura['Model'] == 'NSX'), 'Drive'] = 'RWD'
    acura.loc[(acura['Drive'] == '2WD') | (acura['Drive'] == '2WHDR'), 'Drive'] = 'FWD'
    acura.loc[(acura['Drive'] == '4WD') | (acura['Drive'] == '4WHDR'), 'Drive'] = 'AWD'
    acura.loc[(acura['Drive'].isna()), 'Drive'] = 'FWD'

    # impute number of doors and body style
    acura.loc[(acura['Model'] == 'NSX') & (acura['Model Year'] >= 1995) & (acura['Model Year'] <= 2005), 'Body Style'] = 'Convertible'

    acura.loc[(acura['Model'] == 'MDX'), 'Body Style'] = 'SUV'
    acura.loc[(acura['Model'] == 'RDX'), 'Body Style'] = 'SUV'
    acura.loc[(acura['Model'] == 'ZDX'), 'Body Style'] = 'SUV'
    acura.loc[(acura['Body Style'] == 'SUV'), 'Door'] = '5DR'

    acura.loc[(acura['Model'] == 'TSX') & (acura['Trim'] == 'SPORT WAGON'), 'Door'] = '5DR'
    acura.loc[(acura['Model'] == 'TSX') & (acura['Door'] == '5DR'), 'Body Style'] = 'Stationwagon'

    acura.loc[(acura['Model'] == 'INTEGRA') & (acura['Body Style'] == 'Hatchback') & (acura['Door'] == '4DR'), 'Door'] = '5DR'
    acura.loc[(acura['Model'] == 'INTEGRA') & (acura['Door'] == '5DR'), 'Body Style'] = 'Hatchback'

    acura.loc[(acura['Model'] == 'INTEGRA') & ~((acura['Door'] == '4DR') | (acura['Door'] == '5DR')), 'Door'] = '3DR'

    acura.loc[(acura['Model'] == 'RSX'), 'Door'] = '3DR'

    acura.loc[(acura['Door'] == '2DR'), 'Body Style'] = 'Coupe'
    acura.loc[(acura['Door'] == '3DR'), 'Body Style'] = 'Coupe'
    acura.loc[(acura['Door'] == '4DR'), 'Body Style'] = 'Sedan'

    # filter trim
    trim = ['TYPE R', 'TYPE S', 'GS-R', 'DYNAMIC']
    acura['Trim'] = acura['Trim'].str.extractall('(' + '|'.join(trim) + ')').groupby(level=0)[0].apply(' '.join)
    acura['Trim'] = acura['Trim'].fillna('')

    acura = acura.groupby(['Electric Vehicle Flag', 'Hybrid Vehicle Flag', 'Make', 'Model', 'Model Year', 
                           'Body Style', 'Door', 'Drive', 'Trim'], dropna=False).aggregate('sum').reset_index()

    return acura

In [5]:
acura = get_acura(df)

In [6]:
def get_honda(df):
    df = pd.DataFrame(df)

    # filter vehicles from Honda
    honda = df.loc[(df['Make'].str.match('HONDA'))].copy()
    honda['Make'] = 'HONDA'

    # split model name and trim
    model_names = ['CROSSTOUR', 'ACCORD CROSSTOUR', 'ACCORD', 'ACTY', 'BEAT', 'CRX', 'CIVIC CRX', 
                   'CIVIC DEL SOL', 'CIVIC', 'CLARITY', 'CR-V', 'CR-Z', 'ELEMENT', 'FIT', 'HR-V', 
                   'INSIGHT', 'INTEGRA', 'LIFE', 'ODYSSEY', 'PASSPORT', 'PILOT', 'PRELUDE', 
                   'RIDGELINE', 'S2000', 'STEPWAGON']
    honda['Trim'] = honda['Model'].str.split('(' + '|'.join(model_names) + ')', n=1, expand=True)[2].str.strip()
    honda['Model'] = honda['Model'].str.split('(' + '|'.join(model_names) + ')', n=1, expand=True)[1].str.strip()
    
    # remove nan model names 
    honda = honda.loc[~honda['Model'].isna()]
    
    # adjust model name
    honda.loc[honda['Model'] == 'CRX', 'Model'] = 'CIVIC CRX'
    honda.loc[(honda['Model'] == 'ACCORD CROSSTOUR'), 'Model'] = 'CROSSTOUR'

    # extract and remove driven wheels from Trim
    honda['Drive'] = honda['Trim'].str.extract(r'((2|4|A|F|R)(WD|WHDR))')[0]
    honda['Trim'] = honda['Trim'].str.replace(r'((2|4|A|F|R)(WD|WHDR))', '', regex=True)

    # extract and remove number of doors from Trim
    honda['Door'] = honda['Trim'].str.extract(r'(\dDR)')
    honda['Trim'] = honda['Trim'].str.replace(r'(\dDR)', '', regex=True)

    # extract number of doors from body style
    honda.loc[(honda['Door'].isna()), 'Door'] = honda.loc[honda['Door'].isna(), 'Body Style'].str.extract(r'((Two|Four)Door)')[0]
    honda.loc[(honda['Door'] == 'TwoDoor'), 'Door'] = '2DR'
    honda.loc[(honda['Door'] == 'FourDoor'), 'Door'] = '4DR'

    # impute number of doors and body style
    honda.loc[honda['Model'] == 'STEPWAGON', 'Door'] = '5DR'
    honda.loc[honda['Model'] == 'STEPWAGON', 'Body Style'] = 'Minivan'

    honda.loc[honda['Model'] == 'S2000', 'Door'] = '2DR'
    honda.loc[honda['Model'] == 'S2000', 'Body Style'] = 'Convertible'

    honda.loc[honda['Model'] == 'RIDGELINE', 'Door'] = '4DR'
    honda.loc[honda['Model'] == 'RIDGELINE', 'Body Style'] = 'Truck'

    honda.loc[honda['Model'] == 'PRELUDE', 'Door'] = '2DR'
    honda.loc[honda['Model'] == 'PRELUDE', 'Body Style'] = 'Coupe'

    honda.loc[honda['Model'] == 'PILOT', 'Door'] = '5DR'
    honda.loc[honda['Model'] == 'PILOT', 'Body Style'] = 'SUV'

    honda.loc[honda['Model'] == 'PASSPORT', 'Door'] = '5DR'
    honda.loc[honda['Model'] == 'PASSPORT', 'Body Style'] = 'SUV'

    honda.loc[honda['Model'] == 'ODYSSEY', 'Door'] = '5DR'
    honda.loc[honda['Model'] == 'ODYSSEY', 'Body Style'] = 'Minivan'

    honda.loc[honda['Model'] == 'LIFE', 'Door'] = '5DR'
    honda.loc[honda['Model'] == 'LIFE', 'Body Style'] = 'Hatchback'

    honda.loc[(honda['Model'] == 'INTEGRA') & (honda['Door'] == '2DR'), 'Door'] = '3DR'
    honda.loc[(honda['Model'] == 'INTEGRA') & (honda['Door'] == '3DR'), 'Body Style'] = 'Coupe'
    honda.loc[(honda['Model'] == 'INTEGRA') & (honda['Door'] == '4DR'), 'Body Style'] = 'Sedan'
    
    honda.loc[(honda['Model'] == 'INSIGHT') & (honda['Door'] == '2DR'), 'Door'] = '3DR'
    honda.loc[(honda['Model'] == 'INSIGHT') & (honda['Door'] == '3DR'), 'Body Style'] = 'Coupe'
    honda.loc[(honda['Model'] == 'INSIGHT') & (honda['Door'] == '4DR'), 'Body Style'] = 'Sedan'
    honda.loc[(honda['Model'] == 'INSIGHT') & (honda['Door'] == '5DR'), 'Body Style'] = 'Hatchback'

    honda.loc[honda['Model'] == 'HR-V', 'Door'] = '5DR'
    honda.loc[honda['Model'] == 'HR-V', 'Body Style'] = 'SUV'

    honda.loc[honda['Model'] == 'FIT', 'Door'] = '5DR'
    honda.loc[honda['Model'] == 'FIT', 'Body Style'] = 'Hatchback'

    honda.loc[honda['Model'] == 'ELEMENT', 'Door'] = '5DR'
    honda.loc[honda['Model'] == 'ELEMENT', 'Body Style'] = 'SUV'

    honda.loc[honda['Model'] == 'CROSSTOUR', 'Door'] = '5DR'
    honda.loc[honda['Model'] == 'CROSSTOUR', 'Body Style'] = 'SUV'

    honda.loc[honda['Model'] == 'CR-Z', 'Door'] = '3DR'
    honda.loc[honda['Model'] == 'CR-Z', 'Body Style'] = 'Coupe'

    honda.loc[honda['Model'] == 'CR-V', 'Door'] = '5DR'
    honda.loc[honda['Model'] == 'CR-V', 'Body Style'] = 'SUV'

    honda.loc[honda['Model'] == 'CLARITY', 'Door'] = '4DR'
    honda.loc[honda['Model'] == 'CLARITY', 'Body Style'] = 'Sedan'

    honda.loc[honda['Model'] == 'CIVIC DEL SOL', 'Door'] = '2DR'
    honda.loc[honda['Model'] == 'CIVIC DEL SOL', 'Body Style'] = 'Convertible'

    honda.loc[honda['Model'] == 'CIVIC CRX', 'Door'] = '3DR'
    honda.loc[honda['Model'] == 'CIVIC CRX', 'Body Style'] = 'Coupe'

    honda.loc[honda['Model'] == 'BEAT', 'Door'] = '2DR'
    honda.loc[honda['Model'] == 'BEAT', 'Body Style'] = 'Convertible'
    
    honda.loc[(honda['Model'] == 'ACTY') & (honda['Door'].isna()), 'Door'] = '2DR'
    honda.loc[(honda['Model'] == 'ACTY') & (honda['Door'] == '2DR'), 'Body Style'] = 'Truck'
    honda.loc[(honda['Model'] == 'ACTY') & (honda['Door'] == '4DR'), 'Door'] = '5DR'
    honda.loc[(honda['Model'] == 'ACTY') & (honda['Door'] == '5DR'), 'Body Style'] = 'Minivan'

    honda.loc[(honda['Model'] == 'ACCORD') & 
              (honda['Model Year'] >= 1990) & (honda['Model Year'] <= 1997) & 
              (honda['Trim'].str.contains('WAGON')), 'Door'] = '5DR'
    honda.loc[(honda['Model'] == 'ACCORD') & 
              (honda['Door'] == '5DR'), 'Body Style'] = 'Stationwagon'
    honda.loc[(honda['Model'] == 'ACCORD') & 
              (honda['Model Year'] <= 1989) & 
              ((honda['Body Style'].str.contains('Hatchback')) | 
               (honda['Trim'].str.contains('HATCHBACK'))), 'Door'] = '3DR'
    honda.loc[(honda['Model'] == 'ACCORD') & 
              (honda['Door'] == '3DR'), 'Body Style'] = 'Hatchback'
    honda.loc[(honda['Model'] == 'ACCORD') & (honda['Door'] == '4DR'), 'Body Style'] = 'Sedan'
    honda.loc[(honda['Model'] == 'ACCORD') & (honda['Door'] == '2DR'), 'Body Style'] = 'Coupe'

    honda.loc[(honda['Model'] == 'CIVIC') & 
              (honda['Trim'].str.contains(r'WAGON|WAGOVAN')), 'Door'] = '5DR'
    honda.loc[(honda['Model'] == 'CIVIC') & 
              (honda['Door'] == '5DR'), 'Body Style'] = 'Stationwagon'
    honda.loc[(honda['Model'] == 'CIVIC') & 
              (honda['Model Year'] >= 2017) & 
              (honda['Trim'].str.contains('HATCHBACK')), 'Body Style'] = 'Hatchback'
    honda.loc[(honda['Model'] == 'CIVIC') & 
              (honda['Body Style'] == 'Hatchback'), 'Door'] = '5DR'
    honda.loc[(honda['Model'] == 'CIVIC') & 
              (honda['Model Year'] <= 2005) & 
              (honda['Trim'].str.contains('HATCHBACK')), 'Door'] = '3DR'
    honda.loc[(honda['Model'] == 'CIVIC') & 
              (honda['Door'] == '3DR'), 'Body Style'] = 'Hatchback'
    honda.loc[(honda['Model'] == 'CIVIC') & 
              (honda['Model Year'] >= 1984) & 
              (honda['Body Style'] == 'Hatchback') &
              honda['Door'].isna(), 'Door'] = '3DR'
    honda.loc[(honda['Model'] == 'CIVIC') & 
              (honda['Model Year'] <= 1983) & 
              (honda['Body Style'] == 'Hatchback') & 
              honda['Door'].isna(), 'Door'] = '5DR'
    honda.loc[(honda['Model'] == 'CIVIC') & 
              (honda['Door'] == '2DR'), 'Body Style'] = 'Coupe'
    honda.loc[(honda['Model'] == 'CIVIC') & 
                (honda['Door'] == '4DR'), 'Body Style'] = 'Sedan'

    # impute driven wheels
    honda.loc[(honda['Drive'] == '2WD') | (honda['Drive'] == '2WHDR'), 'Drive'] = 'FWD'
    honda.loc[(honda['Drive'] == '4WD') | (honda['Drive'] == '4WHDR'), 'Drive'] = 'AWD'
    honda.loc[honda['Model'] == 'S2000', 'Drive'] = 'RWD'
    honda.loc[honda['Model'] == 'RIDGELINE', 'Drive'] = 'AWD'
    honda.loc[honda['Drive'].isna(), 'Drive'] = 'FWD'

    # filter trim
    trim = ['TYPE R LE', 'TYPE R', 'SIR', 'SI', 'VTEC']
    honda['Trim'] = honda['Trim'].str.extractall('(' + '|'.join(trim) + ')').groupby(level=0)[0].apply(' '.join)
    honda['Trim'] = honda['Trim'].fillna('')

    # fix and remove trim
    honda.loc[honda['Trim'] == 'SI SI', 'Trim'] = 'SI'
    honda.loc[honda['Model'] == 'PRELUDE', 'Trim'] = ''
    honda.loc[((honda['Model Year'] == 2004) | (honda['Model Year'] == 2005)) &
              (honda['Body Style'] == 'Hatchback') & 
              (honda['Trim'] == 'SI'), 'Trim'] = 'SIR'
    honda.loc[(honda['Trim'] == 'SI') & (honda['Model Year'] <= 2005), 'Trim'] = ''

    # remove outliers
    honda.drop(honda.loc[(honda['Model Year'] == 1992) & 
                         (honda['Model'] == 'CIVIC') & 
                         (honda['Body Style'] == 'Coupe')].index, inplace=True)
    honda.drop(honda.loc[(honda['Model Year'] == 2016) & 
                         (honda['Model'] == 'CIVIC') & 
                         (honda['Trim'] == 'SI')].index, inplace=True)
    honda.drop(honda.loc[(honda['Model Year'] == 2016) & 
                         (honda['Model'] == 'RIDGELINE')].index, inplace=True)
    honda = honda.loc[~honda['Door'].isna()]

    honda = honda.groupby(['Electric Vehicle Flag', 'Hybrid Vehicle Flag', 'Make', 'Model', 'Model Year', 
                           'Body Style', 'Door', 'Drive', 'Trim'], dropna=False).aggregate('sum').reset_index()

    return honda

In [7]:
honda = get_honda(df)

In [8]:
honda_acura = pd.concat([honda, acura])

In [10]:
honda_acura.to_csv('data\honda_acura.csv', index=False)